In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

INFILE = '../data/model_comparison.csv'

In [2]:
df = pd.read_csv(INFILE)
df.head()

,ParticipantID,CompletionCode,ID,EndTime,StartTime,Status,DemographicsTime,Gender,GenderSpecify,AgeBins,...,juv_misd_count,juv_other_count,priors_count,felony,black,married,output,y,FcastTimer,next
0,test,ceoict,2,2020-11-03 18:08:32.139990,2020-11-03 18:03:24.024457,Completed,84.897999,Male,NaN,25-29,...,0.0,0.0,2.0,1.0,1.0,0.0,0.207068,1.0,11.135189,NaN
1,test,ceoict,2,2020-11-03 18:08:32.139990,2020-11-03 18:03:24.024457,Completed,84.897999,Male,NaN,25-29,...,0.0,0.0,0.0,0.0,0.0,0.0,0.381263,0.0,8.531103,NaN
2,test,ceoict,2,2020-11-03 18:08:32.139990,2020-11-03 18:03:24.024457,Completed,84.897999,Male,NaN,25-29,...,0.0,0.0,2.0,1.0,0.0,0.0,0.403128,0.0,9.866797,NaN
3,test,ceoict,2,2020-11-03 18:08:32.139990,2020-11-03 18:03:24.024457,Completed,84.897999,Male,NaN,25-29,...,0.0,1.0,0.0,1.0,1.0,0.0,0.560167,1.0,13.689945,NaN
4,test,ceoict,2,2020-11-03 18:08:32.139990,2020-11-03 18:03:24.024457,Completed,84.897999,Male,NaN,25-29,...,0.0,0.0,7.0,1.0,1.0,0.0,0.710434,1.0,6.015892,NaN


In [3]:
# select participants who passed comprehension checks
df = df[(df.ParticipantID!='test') & (df.FcastComprehension==1) & (df.BonusComprehension==1)]
# select 'actual' forecasts (as opposed to practice forecasts)
df = df[df.Practice==0]
# scale forecasts to be between 0 and 1 (as opposed to 0 and 100)
df['Fcast'] = df['Fcast'] / 100.
# compute DVs
df['Score'] = (df.y - df.Fcast)**2 - (df.y - df.output)**2
df['dFPP'] = (df.Fcast - df.output).loc[(df.y==0)]
df.values.shape

(620, 50)

In [4]:
# not pre-registered cell: added to remove empty values from Score column
df = df.dropna(subset=['Score'])
df.values.shape

(538, 50)

In [5]:
# prediction performance hypothesis
# positive coef on constant regressor indicates model outperformed human predictions
reg = sm.OLS(df.Score, np.array([1]*len(df)))
res = reg.fit().get_robustcov_results(cov_type='cluster', groups=df.ParticipantID)
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Score   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Tue, 03 Nov 2020   Prob (F-statistic):                nan
Time:                        15:30:31   Log-Likelihood:                -27.270
No. Observations:                 538   AIC:                             56.54
Df Residuals:                     537   BIC:                             60.83
Df Model:                           0                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0337      0.012      2.836      0.006       0.010       0.057
==============================================================================
Omnibus:                       27.760   Durbin-Watson:                   1.986
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               34.052
Skew:                           0.482   Prob(JB):                     4.03e-08
Kurtosis:                       3.768   Cond. No.                         1.00
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [6]:
# fairness hypothesis
# positive coef on black indicates that human predictions are less fair than model predictions
no_recid_df = df[df.y == 0]
reg = sm.OLS(no_recid_df.dFPP, no_recid_df.black)
res = reg.fit().get_robustcov_results(cov_type='cluster', groups=no_recid_df.ParticipantID)
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   dFPP   R-squared (uncentered):                   0.030
Model:                            OLS   Adj. R-squared (uncentered):              0.027
Method:                 Least Squares   F-statistic:                              4.228
Date:                Tue, 03 Nov 2020   Prob (F-statistic):                      0.0447
Time:                        15:30:31   Log-Likelihood:                         -39.529
No. Observations:                 249   AIC:                                      81.06
Df Residuals:                     248   BIC:                                      84.57
Df Model:                           1                                                  
Covariance Type:              cluster                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
black          0.0724      0.035      2.056      0.045       0.002       0.143
==============================================================================
Omnibus:                        0.573   Durbin-Watson:                   1.631
Prob(Omnibus):                  0.751   Jarque-Bera (JB):                0.701
Skew:                           0.061   Prob(JB):                        0.704
Kurtosis:                       2.771   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors are robust to cluster correlation (cluster)
"""